# Transformer

Ya tenemos todo el código necesario para crear una clase del transformer entero

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model.png" alt="Transformer architecture" style="width:425px;height:626px;">
</div>

Así que vamos a ello

## Implementación

Escribimos todas las clases de más bajo nivel necesarias

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Embedding(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        """
        Args:
            vocab_size: size of vocabulary
            embed_dim: dimension of embeddings
        """
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

    def forward(self, x):
        """
        Args:
            x: input vector
        Returns:
            out: embedding vector
        """
        return self.embedding(x)

class PositionalEncoding(nn.Module):
    def __init__(self, max_sequence_len, embedding_model_dim):
        """
        Args:
            seq_len: length of input sequence
            embed_model_dim: demension of embedding
        """
        super().__init__()
        self.embedding_dim = embedding_model_dim

        # create constant 'positional_encoding' matrix with values dependant on pos and i
        positional_encoding = torch.zeros(max_sequence_len, self.embedding_dim)
        for pos in range(max_sequence_len):
            for i in range(0, self.embedding_dim, 2):
                positional_encoding[pos, i]     = torch.sin(torch.tensor(pos / (10000 ** ((2 * i) / self.embedding_dim))))
                positional_encoding[pos, i + 1] = torch.cos(torch.tensor(pos / (10000 ** ((2 * (i+1)) / self.embedding_dim))))
        positional_encoding = positional_encoding.unsqueeze(0)
        self.register_buffer('positional_encoding', positional_encoding)

    def forward(self, x):
        """
        Args:
            x: input vector
        Returns:
            x: output
        """
        # make embeddings relatively larger
        x = x * torch.sqrt(torch.tensor(self.embedding_dim))
        
        # add encoding matrix to embedding (x)
        sequence_len = x.size(1)
        # x = x + torch.autograd.Variable(self.positional_encoding[:,:sequence_len], requires_grad=False)
        x = x + self.positional_encoding[:,:sequence_len]
        return x

class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        self.dim_embedding = dim_embedding
    
    def forward(self, query, key, value, mask=None):
        """
        Args:
            key: key vector
            query: query vector
            value: value vector
            mask: mask matrix (optional)
        
        Returns:
            output vector from scaled dot product attention
        """
        # MatMul
        key_trasposed = key.transpose(-1,-2)
        product = torch.matmul(query, key_trasposed)
        # scale
        scale = product / torch.sqrt(torch.tensor(self.dim_embedding))
        # Mask (optional)
        if mask is not None:
            scale = scale.masked_fill(mask == 0, float('-inf'))
        # softmax
        attention_matrix = torch.softmax(scale, dim=-1)
        # MatMul
        output = torch.matmul(attention_matrix, value)
        
        return output

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, dim_embedding):
        """
        Args:
            heads: number of heads
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        
        self.dim_embedding = dim_embedding
        self.dim_proyection = dim_embedding // heads
        self.heads = heads
        
        self.proyection_Q = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_K = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_V = nn.Linear(dim_embedding, dim_embedding)
        self.attention = nn.Linear(dim_embedding, dim_embedding)

        self.scaled_dot_product_attention = ScaledDotProductAttention(self.dim_proyection)
    
    def forward(self, Q, K, V, mask=None):
        """
        Args:
            Q: query vector
            K: key vector
            V: value vector
            mask: mask matrix (optional)

        Returns:
            output vector from multi-head attention
        """
        batch_size = Q.size(0)
        
        # perform linear operation and split into h heads
        proyection_Q = self.proyection_Q(Q).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_K = self.proyection_K(K).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_V = self.proyection_V(V).view(batch_size, -1, self.heads, self.dim_proyection)
        
        # transpose to get dimensions bs * h * sl * d_model
        proyection_Q = proyection_Q.transpose(1,2)
        proyection_K = proyection_K.transpose(1,2)
        proyection_V = proyection_V.transpose(1,2)

        # calculate attention
        scaled_dot_product_attention = self.scaled_dot_product_attention(proyection_Q, proyection_K, proyection_V, mask=mask)
        
        # concatenate heads and put through final linear layer
        concat = scaled_dot_product_attention.transpose(1,2).contiguous().view(batch_size, -1, self.dim_embedding)
        
        output = self.attention(concat)
    
        return output

class AddAndNorm(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding (int): Embedding dimension.
        """
        super().__init__()
        self.normalization = nn.LayerNorm(dim_embedding)

    def forward(self, x, sublayer):
        """
        Args:
            x (torch.Tensor): Input tensor.
            sublayer (torch.Tensor): Sublayer tensor.

        Returns:
            torch.Tensor: Output tensor.
        """
        return self.normalization(torch.add(x, sublayer))

class FeedForward(nn.Module):
    def __init__(self, dim_embedding, increment=4):
        super().__init__()
        self.feed_forward = nn.Sequential(
            nn.Linear(dim_embedding, dim_embedding*increment),
            nn.ReLU(),
            nn.Linear(dim_embedding*increment, dim_embedding)
        )
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        x = self.feed_forward(x)
        return x

class Linear(nn.Module):
    def __init__(self, dim_embedding, vocab_size):
        super().__init__()
        self.linear = nn.Linear(dim_embedding, vocab_size)
        
    def forward(self, x):
        x = self.linear(x)
        return x

class Softmax(nn.Module):
    def __init__(self):
        super().__init__()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.softmax(x)
        return x


Escribimos las clases de medio nivel

In [34]:
class DecoderLayer(nn.Module):
    def __init__(self, heads, dim_embedding):
        """
        Args:
            heads: number of heads
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        self.masked_multi_head_attention = MultiHeadAttention(heads, dim_embedding)
        self.encoder_decoder_multi_head_attention = MultiHeadAttention(heads, dim_embedding)
        self.add_and_norm_1 = AddAndNorm(dim_embedding)
        self.add_and_norm_2 = AddAndNorm(dim_embedding)
        self.add_and_norm_3 = AddAndNorm(dim_embedding)
        self.feed_forward = FeedForward(dim_embedding)
    
    def forward(self, x, encoder_output, mask=None):
        """
        Args:
            x: input vector
            encoder_output: output vector from encoder
            mask: mask matrix (optional)
        
        Returns:
            output vector from decoder layer
        """
        Q = x
        K = x
        V = x
        masked_multi_head_attention = self.masked_multi_head_attention(Q, K, V, mask=mask)
        add_and_norm_1 = self.add_and_norm_1(masked_multi_head_attention, x)

        Q = add_and_norm_1
        K = encoder_output
        V = encoder_output
        encoder_decoder_multi_head_attention = self.encoder_decoder_multi_head_attention(Q, K, V)
        add_and_norm_2 = self.add_and_norm_2(encoder_decoder_multi_head_attention, add_and_norm_1)

        feed_forward = self.feed_forward(add_and_norm_2)
        add_and_norm_3 = self.add_and_norm_3(feed_forward, add_and_norm_2)

        return add_and_norm_3

class Decoder(nn.Module):
    def __init__(self, heads, dim_embedding, Nx):
        """
        Args:
            heads: number of heads
            dim_embedding: dimension of embedding vector
            Nx: number of decoder layers
        """
        super().__init__()
        self.layers = nn.ModuleList([DecoderLayer(heads, dim_embedding) for _ in range(Nx)])
    
    def forward(self, x, encoder_output, mask=None):
        """
        Args:
            x: input vector
            encoder_output: output vector from encoder
            mask: mask matrix (optional)
        
        Returns:
            output vector from decoder
        """
        for decoder_layer in self.layers:
            x = decoder_layer(x, encoder_output, mask)
        return x

class EncoderLayer(nn.Module):
    def __init__(self, heads, dim_embedding):
        super().__init__()
        self.multi_head_attention = MultiHeadAttention(heads, dim_embedding)
        self.add_and_norm_1 = AddAndNorm(dim_embedding)
        self.feed_forward = FeedForward(dim_embedding)
        self.add_and_norm_2 = AddAndNorm(dim_embedding)
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        multi_head_attention = self.multi_head_attention(x, x, x)
        add_and_norm_1 = self.add_and_norm_1(x, multi_head_attention)
        feed_forward = self.feed_forward(add_and_norm_1)
        add_and_norm_2 = self.add_and_norm_2(add_and_norm_1, feed_forward)
        return add_and_norm_2

class Encoder(nn.Module):
    def __init__(self, heads, dim_embedding, Nx):
        super().__init__()
        self.encoder_layers = nn.ModuleList([EncoderLayer(heads, dim_embedding) for _ in range(Nx)])
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(x)
        return x


Y por último escribimos la clase transformer

In [35]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, dim_embedding, src_max_seq_len, tgt_max_seq_len, Nx, heads):
        """
        Args:
            vocab_size: size of vocabulary
            dim_embedding: dimension of embedding vector
            max_sequence_len: maximum length of sequence
            heads: number of heads
            Nx: number of decoder layers
        """
        super().__init__()
        self.encoder = Encoder(heads, dim_embedding, Nx)
        self.decoder = Decoder(heads, dim_embedding, Nx)
        self.sourceEmbedding = Embedding(src_vocab_size, dim_embedding)
        self.targetEmbedding = Embedding(tgt_vocab_size, dim_embedding)
        self.sourcePositional_encoding = PositionalEncoding(src_max_seq_len, dim_embedding)
        self.targetPositional_encoding = PositionalEncoding(tgt_max_seq_len, dim_embedding)
        self.linear = Linear(dim_embedding, tgt_vocab_size)
        self.softmax = Softmax()
    
    def encode(self, source):
        embedding = self.sourceEmbedding(source)
        positional_encoding = self.sourcePositional_encoding(embedding)
        encoder_output = self.encoder(positional_encoding)
        return encoder_output
    
    def decode(self, encoder_output, target, target_mask):
        embedding = self.targetEmbedding(target)
        positional_encoding = self.targetPositional_encoding(embedding)
        decoder_output = self.decoder(positional_encoding, encoder_output, target_mask)
        return decoder_output
    
    def projection(self, decoder_output):
        linear_output = self.linear(decoder_output)
        softmax_output = self.softmax(linear_output)
        return softmax_output
    
    def forward(self, source, target, target_mask):
        # Encode
        embedding_encoder = self.sourceEmbedding(source)
        positional_encoding_encoder = self.sourcePositional_encoding(embedding_encoder)
        encoder_output = self.encoder(positional_encoding_encoder)

        # Decode
        embedding_decoder = self.targetEmbedding(target)
        positional_encoding_decoder = self.targetPositional_encoding(embedding_decoder)
        decoder_output = self.decoder(positional_encoding_decoder, encoder_output, target_mask)

        # Projection
        linear = self.linear(decoder_output)
        softmax = self.softmax(linear)
        return softmax


Rescatamos la función que crea la máscara para el transformer

In [36]:
def create_mask(sequence_len):
    """
    Args:
        sequence_len: length of sequence
        
    Returns:
        mask matrix
    """
    mask = torch.tril(torch.ones((sequence_len, sequence_len)))
    return mask

Volvemos a definir la función que obtiene el embbeding más el positional encoding de BERT

In [37]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

def extract_embeddings(input_sentences, model_name='bert-base-uncased'):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    
    # tokenización de lote
    inputs = tokenizer(input_sentences, return_tensors='pt', padding=True, truncation=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
        
    token_embeddings = outputs[0]
    
    # Los embeddings posicionales están en la segunda capa de los embeddings de la arquitectura BERT
    positional_encodings = model.embeddings.position_embeddings.weight[:token_embeddings.shape[1], :].detach().unsqueeze(0).repeat(token_embeddings.shape[0], 1, 1)

    embeddings_with_positional_encoding = token_embeddings + positional_encodings

    # convierte las IDs de los tokens a tokens
    tokens = [tokenizer.convert_ids_to_tokens(input_id) for input_id in inputs['input_ids']]

    return tokens, inputs['input_ids'], token_embeddings, positional_encodings, embeddings_with_positional_encoding

Creamos una sentencia para el encoder y otra para el decoder

In [38]:
sentence_encoder = "I gave the dog a bone because it was hungry"
sentence_decoder = "Le di un hueso al perro porque tenía hambre"

tokens_encoder, input_ids_encoder, token_embeddings_encoder, positional_encodings_encoder, embeddings_with_positional_encoding_encoder = extract_embeddings(sentence_encoder)
tokens_decoder, input_ids_decoder, token_embeddings_decoder, positional_encodings_decoder, embeddings_with_positional_encoding_decoder = extract_embeddings(sentence_decoder)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.La

In [39]:
token_embeddings_encoder.shape[1], token_embeddings_decoder.shape[1]

(12, 16)

Creamos un objeto `transformer` y obtenemos su salida

In [40]:
src_vocab_size = 30522
tgt_vocab_size = 30522
dim_embedding = max(token_embeddings_encoder.shape[-1], token_embeddings_decoder.shape[-1])
src_max_sequence_len = token_embeddings_encoder.shape[1]
tgt_max_sequence_len = token_embeddings_decoder.shape[1]
heads = 8
Nx = 6
print(f"src vocab_size: {src_vocab_size}, dim_embedding: {dim_embedding}, src max_sequence_len: {src_max_sequence_len}, heads: {heads}, Nx: {Nx}")
print(f"tgt vocab_size: {tgt_vocab_size}, dim_embedding: {dim_embedding}, tgt max_sequence_len: {tgt_max_sequence_len}, heads: {heads}, Nx: {Nx}")
transformer = Transformer(src_vocab_size, tgt_vocab_size, dim_embedding, src_max_sequence_len, tgt_max_sequence_len, Nx, heads)

src vocab_size: 30522, dim_embedding: 768, src max_sequence_len: 12, heads: 8, Nx: 6
tgt vocab_size: 30522, dim_embedding: 768, tgt max_sequence_len: 16, heads: 8, Nx: 6


In [41]:
mask = create_mask(tgt_max_sequence_len)
mask.shape

torch.Size([16, 16])

In [42]:
transformer_output = transformer(input_ids_encoder, input_ids_decoder, target_mask=mask)
transformer_output.shape

torch.Size([1, 16, 30522])

Al igual que en el notebook anterior obtenemos 1x16x30522, es decir 1 batch, 16 tokens y 30522 posibles tokens del vocabulario

**Enhorabuena!!**, si hasta aquí has entendido todo ya tienes lo necesario para implementar un transformer. Pero ahora vamos a ver una última cosa, que son las consideraciones últimas que explican en el paper que han metido para el entrenamiento